# BentoML vs FastAPI: Good-bye FastAPI For Machine Learning!
## Objective comparison between BentoML and FastAPI

### Notes

FastAPI vs BentoML

Similar Features
- Pydantic validation
- Swagger UI
- async
- Starlette

ML Differentiators
- Running models in separate processes
    - https://modelserving.com/blog/breaking-up-with-flask-amp-fastapi-why-ml-model-serving-requires-a-specialized-framework

### Motivation

### But what is an API?

### Saving/loading models
https://docs.bentoml.org/en/latest/concepts/model.html

### Integration with other ML frameworks

https://docs.bentoml.org/en/latest/frameworks/index.html

### Input/Output
https://docs.bentoml.org/en/latest/reference/api_io_descriptors.html

### Dependency management

### Model registry

### Deploying the service
https://docs.bentoml.org/en/latest/concepts/deploy.html

### GPU serving
https://docs.bentoml.org/en/latest/guides/gpu.html

### Batching inputs
https://docs.bentoml.org/en/latest/guides/batching.html